In [2]:
from nlpcda import Ner


In [ ]:
!pip install nlpcda

In [3]:
labels = ['SYMPTOM',
 'DRUG_EFFICACY',
 'PERSON_GROUP',
 'SYNDROME',
 'DRUG_TASTE',
 'DISEASE',
 'DRUG_DOSAGE',
 'DRUG_INGREDIENT',
 'FOOD_GROUP',
 'DISEASE_GROUP',
 'DRUG',
 'FOOD',
 'DRUG_GROUP']

In [5]:
def _cut(sentence):
    """
    将一段文本切分成多个句子
    :param sentence:
    :return:
    """
    new_sentence = []
    sen = []
    for i in sentence:
        if i in ['。', '！', '？', '?'] and len(sen) != 0:
            sen.append(i)
            new_sentence.append("".join(sen))
            sen = []
            continue
        sen.append(i)

    if len(new_sentence) <= 1: # 一句话超过max_seq_length且没有句号的，用","分割，再长的不考虑了。
        new_sentence = []
        sen = []
        for i in sentence:
            if i.split(' ')[0] in ['，', ','] and len(sen) != 0:
                sen.append(i)
                new_sentence.append("".join(sen))
                sen = []
                continue
            sen.append(i)
    if len(sen) > 0:  # 若最后一句话无结尾标点，则加入这句话
        new_sentence.append("".join(sen))
    return new_sentence

def cut_test_set(text_list,len_treshold):
    cut_text_list = []
    cut_index_list = []
    for text in text_list:

        temp_cut_text_list = []
        text_agg = ''
        if len(text) < len_treshold:
            temp_cut_text_list.append(text)
        else:
            sentence_list = _cut(text)  # 一条数据被切分成多句话
            for sentence in sentence_list:
                if len(text_agg) + len(sentence) < len_treshold:
                    text_agg += sentence
                else:
                    temp_cut_text_list.append(text_agg)
                    text_agg = sentence
            temp_cut_text_list.append(text_agg)  # 加上最后一个句子

        cut_index_list.append(len(temp_cut_text_list))
        cut_text_list += temp_cut_text_list

    return cut_text_list, cut_index_list


In [6]:
#设置样本长度
text_length = 250
def from_ann2dic(r_ann_path, r_txt_path, w_path, w_file):
    q_dic = {}
    with codecs.open(r_ann_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip("\n\r")
            line_arr = line.split('\t')
            entityinfo = line_arr[1]
            entityinfo = entityinfo.split(' ')
            cls = entityinfo[0]
            start_index = int(entityinfo[1])
            end_index = int(entityinfo[2])
            length = end_index - start_index
            for r in range(length):
                if r == 0:
                    q_dic[start_index] = ("B-%s" % cls)
                else:
                    q_dic[start_index + r] = ("I-%s" % cls)

    with codecs.open(r_txt_path, "r", encoding="utf-8") as f:
        content_str = f.read()
        
    
    cut_text_list, cut_index_list = cut_test_set([content_str],text_length)
    
    i = 0
    for idx, line in enumerate(cut_text_list):
        w_path_ = "%s/%s-%s-new.txt" % (w_path, w_file,idx)
        with codecs.open(w_path_, "w", encoding="utf-8") as w:
            for str_ in line:
                if str_ is " " or str_ == "" or str_ == "\n" or str_ == "\r":
                    pass
                else:
                    if i in q_dic:
                        tag = q_dic[i]
                    else:
                        tag = "O"  # 大写字母O
                    w.write('%s\t%s\n' % (str_, tag))
                i+=1            

        

In [7]:
import glob
import numpy as np
file_list = glob.glob('./round1_train/train/*.txt') 

In [12]:
import os
import codecs
data_dir = './round1_train/train/'
for file in file_list:
    if file.find(".ann") == -1 and file.find(".txt") == -1:
        continue
    file_name = file.split('/')[-1].split('.')[0]
    r_ann_path = os.path.join(data_dir, "%s.ann" % file_name)
    r_txt_path = os.path.join(data_dir, "%s.txt" % file_name)
    w_path = './round1_train/train_new1/'
    w_file = file_name
    from_ann2dic(r_ann_path, r_txt_path, w_path,w_file)


In [16]:

ner = Ner(ner_dir_name='./round1_train/train_new1/',
        ignore_tag_list=['O'],
        data_augument_tag_list=labels,
        augument_size=3, seed=0)
data_sentence_arrs, data_label_arrs = ner.augment(file_name='./round1_train/train_new1/305-0-new.txt')
# 3条增强后的句子、标签 数据，len(data_sentence_arrs)==3
# 你可以写文件输出函数，用于写出，作为后续训练等
print(data_sentence_arrs, data_label_arrs)

[['孕', '妇', '慎', '用', '。', '口', '服', '。', '一', '次', '5', '粒', '，', '一', '日', '3', '次', '，', '饭', '后', '开', '水', '吞', '服', '或', '遵', '医', '嘱', '。', '尚', '不', '明', '确', '。', '去', '腐', '，', '刺', '激', '精', '子', '生', '成', '。', '用', '于', '急', '、', '慢', '性', '病', '血', '尿', '，', '消', '化', '系', '统', '疾', '病', '后', '错', '，', '妇', '女', '病', '及', '盆', '腔', '炎', '引', '起', '的', '经', '期', '错', '后', '及', '不', '明', '原', '因', '引', '起', '的', '盆', '腔', '炎', '，', '亦', '可', '作', '为', '治', '疗', '肾', '病', '每', '粒', '装', '0', '.', '4', '2', 'g', '江', '苏', '天', '士', '力', '帝', '益', '药', '业', '有', '限', '公', '司', '祛', '湿', '通', '淋', '，', '化', '瘀', '止', '痛', '。', '用', '于', '急', '、', '盆', '腔', '炎', '血', '尿', '，', '列', '腺', '增', '生', '血', '尿', '，', '子', '宫', '肌', '瘤', '及', '感', '冒', '引', '起', '的', '腰', '膝', '酸', '软', '及', '不', '明', '原', '因', '引', '起', '的', '妊', '娠', '斑', '，', '亦', '可', '作', '为', '治', '疗', '癌', '症', '的', '辅', '助', '药', '物', '。'], ['膀', '胱', '气', '化', '不', '行', '者', '慎', '用', '。', '口', '服', '。', '一', '

In [17]:
len(data_sentence_arrs)

3

In [18]:
for i,j in zip(data_sentence_arrs[0],data_label_arrs[0]):
    print(i,j)

孕 B-PERSON_GROUP
妇 I-PERSON_GROUP
慎 O
用 O
。 O
口 O
服 O
。 O
一 O
次 O
5 O
粒 O
， O
一 O
日 O
3 O
次 O
， O
饭 O
后 O
开 O
水 O
吞 O
服 O
或 O
遵 O
医 O
嘱 O
。 O
尚 O
不 O
明 O
确 O
。 O
去 B-DRUG_EFFICACY
腐 I-DRUG_EFFICACY
， O
刺 B-DRUG_EFFICACY
激 I-DRUG_EFFICACY
精 I-DRUG_EFFICACY
子 I-DRUG_EFFICACY
生 I-DRUG_EFFICACY
成 I-DRUG_EFFICACY
。 O
用 O
于 O
急 O
、 O
慢 B-DISEASE_GROUP
性 I-DISEASE_GROUP
病 I-DISEASE_GROUP
血 O
尿 O
， O
消 B-DISEASE_GROUP
化 I-DISEASE_GROUP
系 I-DISEASE_GROUP
统 I-DISEASE_GROUP
疾 I-DISEASE_GROUP
病 I-DISEASE_GROUP
后 B-SYMPTOM
错 I-SYMPTOM
， O
妇 B-DISEASE_GROUP
女 I-DISEASE_GROUP
病 I-DISEASE_GROUP
及 O
盆 B-DISEASE
腔 I-DISEASE
炎 I-DISEASE
引 O
起 O
的 O
经 B-SYMPTOM
期 I-SYMPTOM
错 I-SYMPTOM
后 I-SYMPTOM
及 O
不 O
明 O
原 O
因 O
引 O
起 O
的 O
盆 B-SYMPTOM
腔 I-SYMPTOM
炎 I-SYMPTOM
， O
亦 O
可 O
作 O
为 O
治 O
疗 O
肾 B-DISEASE_GROUP
病 I-DISEASE_GROUP
每 O
粒 O
装 O
0 O
. O
4 O
2 O
g O
江 O
苏 O
天 O
士 O
力 O
帝 O
益 O
药 O
业 O
有 O
限 O
公 O
司 O
祛 B-DRUG_EFFICACY
湿 I-DRUG_EFFICACY
通 I-DRUG_EFFICACY
淋 I-DRUG_EFFICACY
， O
化 B-DRUG_EFFICACY
瘀 I-

In [19]:
"".join(data_sentence_arrs[0])

'孕妇慎用。口服。一次5粒，一日3次，饭后开水吞服或遵医嘱。尚不明确。去腐，刺激精子生成。用于急、慢性病血尿，消化系统疾病后错，妇女病及盆腔炎引起的经期错后及不明原因引起的盆腔炎，亦可作为治疗肾病每粒装0.42g江苏天士力帝益药业有限公司祛湿通淋，化瘀止痛。用于急、盆腔炎血尿，列腺增生血尿，子宫肌瘤及感冒引起的腰膝酸软及不明原因引起的妊娠斑，亦可作为治疗癌症的辅助药物。'